In [9]:
import gensim
import re

x_train = open('/content/drive/MyDrive/Colab Notebooks/film_plots_lemma.txt',
               encoding='utf-8').read().split('\n')

y_train = open('/content/drive/MyDrive/Colab Notebooks/wiki_titles.txt',
               encoding='utf-8').read().split('\n')


x_test = open('/content/drive/MyDrive/Colab Notebooks/test_data_lemma.txt',
              encoding='utf-8').read().split('\n')

y_test = open('/content/drive/MyDrive/Colab Notebooks/test_titles.txt',
              encoding='utf-8').read().split('\n')

x_summary_test = open('/content/drive/MyDrive/Colab Notebooks/summary_lemma.txt',
                      encoding='utf-8').read().split('\n')

y_summary_test = open('/content/drive/MyDrive/Colab Notebooks/summary_titles.txt',
                      encoding='utf-8').read().split('\n')

print(len(x_train) == len(y_train))
print(len(x_test) == len(y_test))
print(len(x_summary_test) == len(y_summary_test))

x_train = [re.findall('\w+', text) for text in x_train]
x_test = [re.findall('\w+', text) for text in x_test]
x_summary_test = [re.findall('\w+', text) for text in x_summary_test]

model = gensim.models.KeyedVectors.load('/content/drive/MyDrive/Colab Notebooks/model.model') # tayga_none_fasttextcbow_300_10_2019

True
True
True


In [37]:
import pandas as pd


def ap(relev, k):    # average presicion
    ap = []
    for i in range(1, k):
        if relev[i-1] is 1:    # if doc is relevant
            ap.append(sum(relev[:i])/i)    # summary of precisions
    try: 
        ap = sum(ap)/sum(relev)
    except ZeroDivisionError:
        ap = 0.0
    return ap


def evaluation(query, relev, index=0, k=5):    # mean average precision (5)
    prec = round(sum(relev)/k, 4)    # rank is not take into account
    k += 1
    avp = ap(relev, k)    # rank is take into account
    evaluat = pd.DataFrame({'precision': prec,
                            'average_precision': avp},
                            index=[index])
    return evaluat


def calc_map(query, film, predictions, k, index):
    relev = [0] * k
    for i, pred in enumerate(predictions):
        if y_train[pred] == film:
            relev[i] = 1
    if relev != [0] * k:   
        df = evaluation(query, relev, index, k)
        print(index)
    else:
        df = pd.DataFrame({'precision': 0.0,
                           'average_precision': 0.0},
                           index=[index])
    return df


# map = sum(ap)/Q    # Q - number of quaries
# recall = sum(p)/Q

In [3]:
pip install annoy

In [ ]:
from annoy import AnnoyIndex # https://github.com/spotify/annoy
import numpy as np

f = 300
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
for idx, text in enumerate(x_train):
  vectors = []
  for word in text:
    try:
      vectors.append(model.__getitem__(word))
    except AttributeError:
      continue
  v = np.mean(vectors, axis=0)
  try:
    t.add_item(idx, v)
  except ValueError:
    continue

t.build(100) # 100 trees
t.save('test.ann')

In [ ]:
u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, just mmap the file

df = pd.DataFrame(columns=['precision', 'average_precision'])

for idx, (query, film) in enumerate(zip(x_test, y_test)):
  vectors = []
  for word in query:
    try:
      vectors.append(model.__getitem__(word))
    except AttributeError:
      continue
  v = np.mean(vectors, axis=0)
  top_20 = u.get_nns_by_vector(v, 20) # find the 20 nearest neighbors

  df2 = calc_map(query, film, top_20, 20, idx)
  df = df.append(df2)

In [39]:
MAP = pd.DataFrame(columns=['recall', 'MAP'])
df2 = pd.DataFrame({'recall': len([1 for prec in df.precision if prec != 0])/len(x_test),
                    'MAP': sum(df.average_precision)/len(x_test)},
                    index=['vk'])
MAP = MAP.append(df2)
MAP

,recall,MAP
vk,0.049,0.01778


In [40]:
indexes = [28, 89, 110, 124, 129, 148, 151, 190, 208, 219, 252,
           293, 329, 341, 353, 363, 368, 370, 374, 376, 377, 380,
           381, 395, 407, 411, 428, 466, 470, 471, 472, 486, 610,
           611, 636, 638, 648, 651, 652, 654, 655, 667, 678, 691,
           762, 858, 879, 888, 982]
len(indexes)

49

In [33]:
df = pd.DataFrame(columns=['precision', 'average_precision'])

for idx, (query, film) in enumerate(zip(x_summary_test[:10], y_summary_test[:10])):
  vectors = []
  for word in query:
    try:
      vectors.append(model.__getitem__(word))
    except AttributeError:
      continue
  v = np.mean(vectors, axis=0)
  top_20 = u.get_nns_by_vector(v, 20) # find the 20 nearest neighbors

  df2 = calc_map(query, film, top_20, 20, idx)
  df = df.append(df2)

In [34]:
df2 = pd.DataFrame({'recall': len([1 for prec in df.precision if prec != 0])/len(x_summary_test[:10]),
                    'MAP': sum(df.average_precision)/len(x_summary_test[:10])},
                    index=['wiki summaries'])
MAP = MAP.append(df2)
MAP

,recall,MAP
vk,0.0,0.0
wiki summaries,0.0,0.0


In [28]:
# check if model works correctly

u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, just mmap the file 

for idx, text in enumerate(x_train[:5]):
  vectors = []
  for word in text:
    try:
      vectors.append(model.__getitem__(word))
    except AttributeError:
      continue
  v = np.mean(vectors, axis=0)
  top_20 = u.get_nns_by_vector(v, 20) # find the 20 nearest neighbors

  print(top_20[0] == idx)

True
True
True
True
True
